In [1]:
def factorial(n):
    if n in [0, 1]:
        return 1
    else:
        r = 1
        for i in range(2, n+1):
            r *= i
        return r

# число сочетаний из N по K, то есть число подмножеств размера K у множества размера N
def C(n, k):
    return factorial(n) // factorial(n - k) // factorial(k)

In [2]:
import itertools

cell_count = 9   # всего клеток на поле
options = 'xo '  # варианты - крестик, нолик или пусто
upper_limit = len(options) ** cell_count
print("Верхняя граница оценки числа позиций игры в крестики-нолики =", upper_limit)

upper_limit = 0
for occupied_count in range(cell_count+1):
    o_s = occupied_count // 2
    upper_limit += C(cell_count, occupied_count) * C(occupied_count, o_s)

print("Более точная оценка числа позиций =", upper_limit)

Верхняя граница оценки числа позиций игры в крестики-нолики = 19683
Более точная оценка числа позиций = 6046


In [3]:
# выигрышные позиции
def wins(field, who):
    combinations = [[0, 1, 2], [3, 4, 5], [6, 7, 8],
                    [0, 3, 6], [1, 4, 7], [2, 5, 8],
                    [0, 4, 8], [2, 4, 6]]
    # для каждой комбинации проверяем, что её занимают 3 символа 'who'
    return any(map(lambda pos: list(map(lambda p: field[p], pos)).count(who) == 3,  combinations))

# позиция могла возникнуть у ходе игры
def is_valid(field):
    xs, os, _s = field.count('x'), field.count('o'), field.count(' ')
    # не может быть такого, что оба победили
    both_wins = wins(field, 'x') and wins(field, 'o')
    # клеток 9, всех сколько нужно
    return len(field) == 9 and (xs + os + _s == 9) and (xs == os or xs == os + 1) and not both_wins

# печать поля
def print_field(fld):
    print(fld[:3].replace(' ', '.'))
    print(fld[3:6].replace(' ', '.'))
    print(fld[6:].replace(' ', '.'))
    print()

# сгенерируем все возможные поля и отберём только корректные варианты
fields = list(map(lambda t: ''.join(t), filter(is_valid, itertools.product(options, repeat=9))))
print("Оказывается, легальных конфигураций поля =", len(fields))

Оказывается, легальных конфигураций поля = 5890


In [4]:
xwinset = set()
owinset = set()
tieset = set()
for field in fields:
    if wins(field, 'x'):
        xwinset.add(field)
    elif wins(field, 'o'):
        owinset.add(field)
    else:
        tieset.add(field)

print("Победа x =", len(xwinset), "; Победа o =", len(owinset), "; [Пока] ничья =", len(tieset))

Победа x = 942 ; Победа o = 412 ; [Пока] ничья = 4536


In [5]:
# функция проверяет, может ли позиция bigger получиться из позиции smaller
def is_subfield(bigger, smaller):
    return all(map(lambda a, b: a == b or b == ' ', bigger, smaller))

# функция проверяет, может ли позиция bigger получиться из позиции smaller за 1 ход
def is_one_move_ahead(bigger, smaller):
    return sum(map(lambda a, b: a != b and b == ' ', bigger, smaller)) == 1

In [6]:
# построим граф переходов перебором "в лоб"
# ВНИМАНИЕ, самая долгая операция, может занять несколько минут
# построить такой граф можно быстрее. Подумайте как
def build_graph(fields):
    graph = dict()
    for field1 in fields:
        for field2 in fields:
            if is_subfield(field1, field2) and is_one_move_ahead(field1, field2):
                if field2 not in graph:
                    graph[field2] = set()
                graph[field2].add(field1)
    return graph

graph = build_graph(fields)

In [7]:
# посмотрим на N ходов вперёд, получим всех "наследников" поля field
def get_N_moves_descendants(field, N):
    if field not in graph:
        return set()
    descendants = set(graph[field])
    for i in range(N-1):
        new_descendants = set()
        for descendant in descendants:
            if descendant in graph:
                new_descendants |= graph[descendant]
        descendants = new_descendants
    return set(descendants) # чтобы не испортить сам граф - делаем копию

In [8]:
# ищем ответный ход для игрока who. Достаточно простая стратегия
def find_answer(field, who):
    # если играет крестик - его цель xwinset
    target_set = owinset if who == 'o' else xwinset
    afraid_set = xwinset if who == 'o' else owinset
    # среди непосредственных потомков ищем победные ходы
    winning = target_set & get_N_moves_descendants(field, 1)
    if winning:
        return list(winning)[0]
    else:
        candidates = get_N_moves_descendants(field, 1)
        #1 удалим плохие ходы, которые приведут к поражению от следующего хода противника
        problems = get_N_moves_descendants(field, 2) & afraid_set
        to_remove = set()
        for p in problems:
            to_remove |= set(filter(lambda candidate: is_subfield(p, candidate), candidates))
        candidates -= to_remove
        # если ходов не осталось, выходим разочарованно
        print("Осталось", len(candidates), "относительно безопасных ходов")
        if not candidates:
            return "Сдаюсь"
        # дальше начинается "стратегия". Выберем простую.
        # Для каждого хода посчитаем число победных позиций и вычтем число поражений
        wins = dict()
        losses = dict()
        for candidate in candidates:
            # пока доска не наполнится
            wins[candidate] = set()
            losses[candidate] = set()
            for x in range(3, cell_count+2, 2):
                wins[candidate] |= (target_set & get_N_moves_descendants(candidate, x))
            # пока доска не наполнится
            for x in range(2, cell_count+2, 2):
                losses[candidate] |= (afraid_set & get_N_moves_descendants(candidate, x))
        r = sorted(candidates, key=lambda c: len(wins[c]) - len(losses[c]), reverse=True)
        print("Ожидаемых побед", len(wins[r[0]]), "; Ожидаемых поражений", len(losses[r[0]]))
        print("Ход", who, ":")
        return r[0]

In [9]:
# предложим алгоритму сыграть самому собой
field = '         '
while field != "Сдаюсь" and field.count(' ') > 0:
    print_field(field)
    who = 'x' if field.count(' ') % 2 == 1 else 'o'
    winset = xwinset if who == 'x' else owinset
    field = find_answer(field, who)
    if field in winset:
        print("ПОБЕДА!")
        print_field(field)
        break
else:
    if field != "Сдаюсь":
        print_field(field)
    else:
        print("Противник сдался")

...
...
...

Осталось 9 относительно безопасных ходов
Ожидаемых побед 184 ; Ожидаемых поражений 24
Ход x :
...
.x.
...

Осталось 8 относительно безопасных ходов
Ожидаемых побед 12 ; Ожидаемых поражений 76
Ход o :
...
.x.
o..

Осталось 7 относительно безопасных ходов
Ожидаемых побед 37 ; Ожидаемых поражений 4
Ход x :
...
.x.
o.x

Осталось 1 относительно безопасных ходов
Ожидаемых побед 4 ; Ожидаемых поражений 4
Ход o :
o..
.x.
o.x

Осталось 1 относительно безопасных ходов
Ожидаемых побед 2 ; Ожидаемых поражений 0
Ход x :
o..
xx.
o.x

Осталось 1 относительно безопасных ходов
Ожидаемых побед 0 ; Ожидаемых поражений 0
Ход o :
o..
xxo
o.x

Осталось 3 относительно безопасных ходов
Ожидаемых побед 0 ; Ожидаемых поражений 0
Ход x :
o..
xxo
oxx

Осталось 1 относительно безопасных ходов
Ожидаемых побед 0 ; Ожидаемых поражений 0
Ход o :
oo.
xxo
oxx

Осталось 1 относительно безопасных ходов
Ожидаемых побед 0 ; Ожидаемых поражений 0
Ход x :
oox
xxo
oxx

